# 7. LangSmith를 활용한 RAG 애플리케이션 평가


In [1]:
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = userdata.get("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_PROJECT"] = "agent-book"

## 7.4. Ragas를 활용한 합성 테스트 데이터 생성


### 패키지 설치


In [2]:
!pip install langchain-core==0.2.30 langchain-openai==0.1.21 \
    langchain-community==0.2.12 GitPython==3.1.43 \
    langchain-chroma==0.1.2 chromadb==0.5.3 \
    ragas==0.1.14 nest-asyncio==1.6.0 pydantic==2.9.2 numpy==1.25.2

### 검색 대상 문서 로드


In [3]:
from langchain_community.document_loaders import GitLoader


def file_filter(file_path: str) -> bool:
    return file_path.endswith(".mdx")


loader = GitLoader(
    clone_url="https://github.com/langchain-ai/langchain",
    repo_path="./langchain",
    branch="master",
    file_filter=file_filter,
)

documents = loader.load()
print(len(documents))

412


### Ragas를 활용한 합성 테스트 데이터 생성 구현


In [4]:
for document in documents:
    document.metadata["filename"] = document.metadata["source"]

#### 【주의】알려진 오류에 관해

아래 코드에서 gpt-4o를 사용할 경우 OpenAI API의 Usage tier에 따라 RateLimitError가 발생할 수 있다고 보고되었습니다.

OpenAI API의 Usage tier에 관해서는 공식 문서의 다음 페이지를 참조하세요.

https://platform.openai.com/docs/guides/rate-limits/usage-tiers

이 오류가 발생한 경우 다음 중 하나의 대응을 실시하세요.

1. 같은 Tier에서도 gpt-4o보다 레이트 리밋이 높은 gpt-4o-mini를 사용한다
   - 이 경우, 생성되는 합성 테스트 데이터의 품질이 낮아질 것으로 예상됩니다
2. 과금 등을 통해 Tier를 올린다
   - Tier 2에서는 RateLimitError가 발생하지 않는 것을 확인했습니다 (2024년 10월 31일 기준)

##### 2025/3/15 추가

LangChain 문서의 증가로 인해, gpt-4o-mini를 사용하더라도 Tier 1에서는 오류가 발생한다는 보고가 있습니다.

이 경우, GitHub에서 문서를 로드하는 부분에서 다음과 같이 작동이 확인된 버전인 `langchain==0.2.13`을 지정하도록 하세요.

```python
loader = GitLoader(
    clone_url="https://github.com/langchain-ai/langchain",
    repo_path="./langchain",
    branch="langchain==0.2.13",
    file_filter=file_filter,
)
```


In [5]:
import nest_asyncio
from ragas.testset.generator import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

nest_asyncio.apply()

generator = TestsetGenerator.from_langchain(
    generator_llm=ChatOpenAI(model="gpt-4o"),
    critic_llm=ChatOpenAI(model="gpt-4o"),
    embeddings=OpenAIEmbeddings(),
)

testset = generator.generate_with_langchain_docs(
    documents,
    test_size=4,
    distributions={simple: 0.5, reasoning: 0.25, multi_context: 0.25},
)

embedding nodes:   0%|          | 0/1274 [00:00<?, ?it/s]

Generating:   0%|          | 0/4 [00:00<?, ?it/s]

In [6]:
testset.to_pandas()

,question,contexts,ground_truth,evolution_type,metadata,episode_done
0,What are the steps involved in the installatio...,[# RWKV-4\n\nThis page covers how to use the `...,The steps involved in the installation and set...,simple,[{'source': 'docs/docs/integrations/providers/...,True
1,What makes Neo4j an effective open-source data...,[# Neo4j\n\n>What is `Neo4j`?\n\n>- Neo4j is a...,Neo4j is an effective open-source database man...,simple,[{'source': 'docs/docs/integrations/providers/...,True
2,How do `rate_limiter` & `max_retries` help wit...,[ perform tasks such as fetching data from a d...,The `rate_limiter` parameter helps control the...,reasoning,[{'source': 'docs/docs/concepts/chat_models.md...,True
3,How do URL formats for high-availability Redis...,[# Redis\n\n>[Redis (Remote Dictionary Server)...,The context provides information on the URL fo...,multi_context,[{'source': 'docs/docs/integrations/providers/...,True


### LangSmith의 Dataset 생성


In [7]:
from langsmith import Client

dataset_name = "agent-book"

client = Client()

if client.has_dataset(dataset_name=dataset_name):
    client.delete_dataset(dataset_name=dataset_name)

dataset = client.create_dataset(dataset_name=dataset_name)

### 합성 테스트 데이터 저장


In [8]:
inputs = []
outputs = []
metadatas = []

for testset_record in testset.test_data:
    inputs.append(
        {
            "question": testset_record.question,
        }
    )
    outputs.append(
        {
            "contexts": testset_record.contexts,
            "ground_truth": testset_record.ground_truth,
        }
    )
    metadatas.append(
        {
            "source": testset_record.metadata[0]["source"],
            "evolution_type": testset_record.evolution_type,
        }
    )

In [9]:
client.create_examples(
    inputs=inputs,
    outputs=outputs,
    metadata=metadatas,
    dataset_id=dataset.id,
)

## 7.5. LangSmith와 Ragas를 활용한 오프라인 평가 구현


### 커스텀 Evaluator 구현


In [10]:
from typing import Any

from langchain_core.embeddings import Embeddings
from langchain_core.language_models import BaseChatModel
from langsmith.schemas import Example, Run
from ragas.embeddings import LangchainEmbeddingsWrapper
from ragas.llms import LangchainLLMWrapper
from ragas.metrics.base import Metric, MetricWithEmbeddings, MetricWithLLM


class RagasMetricEvaluator:
    def __init__(self, metric: Metric, llm: BaseChatModel, embeddings: Embeddings):
        self.metric = metric

        # LLM과 Embeddings을 Metric에 설정
        if isinstance(self.metric, MetricWithLLM):
            self.metric.llm = LangchainLLMWrapper(llm)
        if isinstance(self.metric, MetricWithEmbeddings):
            self.metric.embeddings = LangchainEmbeddingsWrapper(embeddings)

    def evaluate(self, run: Run, example: Example) -> dict[str, Any]:
        context_strs = [doc.page_content for doc in run.outputs["contexts"]]

        # Ragas의 평가 메트릭의 score 메서드로 점수 산출
        score = self.metric.score(
            {
                "question": example.inputs["question"],
                "answer": run.outputs["answer"],
                "contexts": context_strs,
                "ground_truth": example.outputs["ground_truth"],
            },
        )
        return {"key": self.metric.name, "score": score}

In [11]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from ragas.metrics import answer_relevancy, context_precision

metrics = [context_precision, answer_relevancy]

llm = ChatOpenAI(model="gpt-4o", temperature=0)
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

evaluators = [
    RagasMetricEvaluator(metric, llm, embeddings).evaluate
    for metric in metrics
]

### 추론 함수 구현


In [12]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
db = Chroma.from_documents(documents, embeddings)

In [13]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_template('''\
다음 문맥만을 고려해 질문에 답하세요.

문맥: """
{context}
"""

질문: {question}
''')

model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

retriever = db.as_retriever()

chain = RunnableParallel(
    {
        "question": RunnablePassthrough(),
        "context": retriever,
    }
).assign(answer=prompt | model | StrOutputParser())

In [14]:
def predict(inputs: dict[str, Any]) -> dict[str, Any]:
    question = inputs["question"]
    output = chain.invoke(question)
    return {
        "contexts": output["context"],
        "answer": output["answer"],
    }

### 오프라인 평가 구현·실행


In [18]:
from langsmith.evaluation import evaluate

evaluate(
    predict,
    data="agent-book",
    evaluators=evaluators,
)

View the evaluation results for experiment: 'terrific-farm-92' at:
https://smith.langchain.com/o/ad054265-a6bb-5f53-a2f9-4a6896a6a728/datasets/5389bc1d-5138-4636-bc68-f482ae5e9247/compare?selectedSessions=c74e667a-98f5-45de-89c2-6c806ae46b55




0it [00:00, ?it/s]

,inputs.question,outputs.contexts,outputs.answer,error,reference.contexts,reference.ground_truth,feedback.context_precision,feedback.answer_relevancy,execution_time,example_id,id
0,What are the steps involved in the installatio...,[page_content='# Lantern\n\nThis page covers h...,문맥에는 RWKV-4 wrapper에 대한 정보가 포함되어 있지 않습니다. 따라서 ...,None,[# RWKV-4\n\nThis page covers how to use the `...,The steps involved in the installation and set...,0.000000,0.000000,1.820270,b92d80e4-c378-4aef-bbc8-a6cb168086d2,a1801fa8-014c-434f-889a-6f05c1dd4b6b
1,How do `rate_limiter` & `max_retries` help wit...,[page_content='# MODEL_RATE_LIMIT\n\nYou have ...,The `rate_limiter` and `max_retries` parameter...,None,[ perform tasks such as fetching data from a d...,The `rate_limiter` parameter helps control the...,0.833333,0.974888,4.093221,1e0a5cbf-111c-4a6b-9a1b-953af7c7dd41,13c7850c-7af5-4beb-8515-f439eb969bd3
2,What makes Neo4j an effective open-source data...,[page_content='# Neo4j\n\n>What is `Neo4j`?\n\...,Neo4j is an effective open-source database man...,None,[# Neo4j\n\n>What is `Neo4j`?\n\n>- Neo4j is a...,Neo4j is an effective open-source database man...,1.000000,0.972097,4.521203,291ea723-0e0e-479b-aa07-8a10478409b0,c752a924-5249-4f14-aff6-6d146065e700
3,How do URL formats for high-availability Redis...,[page_content='# Redis\n\n>[Redis (Remote Dict...,The URL formats for high-availability Redis di...,None,[# Redis\n\n>[Redis (Remote Dictionary Server)...,The context provides information on the URL fo...,0.000000,0.981162,6.380748,ee0d61b4-8439-4f64-a45d-5569f3960c6c,aadea6f5-015b-4a94-9cb4-f2f6155d3310


## LangSmith를 활용한 온라인 평가 구현


### 피드백 버튼을 표시하는 함수 구현


In [16]:
from uuid import UUID

import ipywidgets as widgets
from IPython.display import display
from langsmith import Client


def display_feedback_buttons(run_id: UUID) -> None:
    # Good 버튼과 Bad 버튼 준비
    good_button = widgets.Button(
        description="Good",
        button_style="success",
        icon="thumbs-up",
    )
    bad_button = widgets.Button(
        description="Bad",
        button_style="danger",
        icon="thumbs-down",
    )

    # 클릭 시 실행될 함수 정의
    def on_button_clicked(button: widgets.Button) -> None:
        if button == good_button:
            score = 1
        elif button == bad_button:
            score = 0
        else:
            raise ValueError(f"Unknown button: {button}")

        client = Client()
        client.create_feedback(run_id=run_id, key="thumbs", score=score)
        print("피드백을 전송했습니다")

    # 버튼 클릭 시 on_button_clicked 함수 실행
    good_button.on_click(on_button_clicked)
    bad_button.on_click(on_button_clicked)

    # 버튼 표시
    display(good_button, bad_button)

### 피드백 버튼 표시


In [20]:
from langchain_core.tracers.context import collect_runs

# LangSmith의 트레이스 ID(Run ID)를 가져오기 위해 collect_runs 함수 사용
with collect_runs() as runs_cb:
    output = chain.invoke("LangChain의 개요를 알려줘")
    print(output["answer"])
    run_id = runs_cb.traced_runs[0].id

display_feedback_buttons(run_id)

LangChain은 대형 언어 모델(LLM)을 기반으로 한 애플리케이션 개발을 위한 프레임워크입니다. 이 프레임워크는 LLM 애플리케이션의 라이프사이클의 모든 단계를 간소화합니다. 주요 기능은 다음과 같습니다:

1. **개발**: LangChain의 오픈 소스 구성 요소와 제3자 통합을 사용하여 애플리케이션을 구축할 수 있습니다. LangGraph를 사용하여 상태를 유지하는 에이전트를 만들고, 스트리밍 및 인간 개입 지원을 제공합니다.

2. **생산화**: LangSmith를 사용하여 애플리케이션을 검사, 모니터링 및 평가하여 지속적으로 최적화하고 자신 있게 배포할 수 있습니다.

3. **배포**: LangGraph 애플리케이션을 프로덕션 준비가 완료된 API 및 어시스턴트로 변환할 수 있습니다.

LangChain은 대형 언어 모델 및 관련 기술(예: 임베딩 모델 및 벡터 저장소)에 대한 표준 인터페이스를 구현하고 있으며, 수백 개의 공급자와 통합됩니다. 이 프레임워크는 여러 오픈 소스 라이브러리로 구성되어 있으며, 다양한 구성 요소를 조합하여 복잡한 애플리케이션을 구축할 수 있도록 지원합니다.


Button(button_style='success', description='Good', icon='thumbs-up', style=ButtonStyle())

Button(button_style='danger', description='Bad', icon='thumbs-down', style=ButtonStyle())

피드백을 전송했습니다
